In [1]:
import veloxchem as vlx

### Define a VeloxChem Molecule for which the interpolation mechaics force field (IM-MM) shall be constructed.

In [2]:
# Initialize the Molecu
molecule_xyz = '''16

C              1.354430000000         0.723890000000         0.062850000000
C              0.075030000000         1.193180000000        -0.131980000000
S             -1.063880000000        -0.076840000000        -0.238910000000
C              0.145050000000        -1.275960000000        -0.023130000000
C              1.392450000000        -0.699780000000         0.133940000000
C             -0.161190000000        -2.684380000000         0.016020000000
C              0.347150000000        -3.612800000000         0.907410000000
C             -0.139800000000        -4.931010000000         0.664450000000
C             -0.992970000000        -4.972470000000        -0.413030000000
S             -1.212210000000        -3.430270000000        -1.117120000000
H              2.219640000000         1.368740000000         0.151510000000
H             -0.248470000000         2.221020000000        -0.218570000000
H              2.299910000000        -1.273800000000         0.280250000000
H              1.028740000000        -3.357480000000         1.709320000000
H              0.125220000000        -5.802850000000         1.250990000000
H             -1.502580000000        -5.833680000000        -0.824080000000
'''


molecule = vlx.Molecule.from_xyz_string(molecule_xyz)
molecule.show(atom_indices=False)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

### Initialize the QM-Driver determining the underlying quality of the interpolation data.

In [3]:
# Initialize the basis set and the QM driver
basis_set_label = "def2-svp"
basis = vlx.MolecularBasis.read(molecule, basis_set_label)
qm_driver = vlx.ScfRestrictedDriver()
qm_driver.xcfun = "b3lyp"

##### Define dyanmics and interpolation settings which are crucial for the quality in timing of the database. Important variables:

- .desired_point_density -> how many datapoints should the database contain for each conformer.
- .converged_cycle -> after how many cycles the construction for 1 conformer terminates (1 cycle: 3000 steps without database expansion).
- .energy_thrsh -> allowed devaition of the IM energy and QM reference calculations.

##### As this notebook is intended to be a practival example on how to use the IMFroceFieldGenerator but keeping the computational effort at a reasoable level, the shown settings have been used. For production run we recommend using the following settings:
 
 - .nsteps = 50000
 - .snapshots = 10000
 - .desired_point_density = 80
 - .converged_cycle = 5
 - .energy_threshold = 2.0 kcal/mol

In [ ]:
# Initialize the IM force field generator with:
# dihedrals: list of dihedral angles to sample
# sampling_structures: number of structures to sample
# ensemble: NVT or NVE
# temperature: temperature of the ensemble
# nsteps: number of steps to run the dynamics
# snapshots: number of snapshots to save

dyn_drv = vlx.IMForceFieldGenerator(qm_driver)
dyn_drv.dihedrals_dict = [((2, 3, 5, 9), 10)]
dyn_drv.ensemble = 'NVT'
dyn_drv.temperature = 250
dyn_drv.nsteps = 10000
dyn_drv.snapshots = 5000
dyn_drv.converged_cycle = 3
dyn_drv.energy_threshold = 2.0
dyn_drv.desired_point_density = 10
dyn_drv.nstruc_to_confirm_database_quality = 10
dyn_drv.minimize = False


IMPORTANT: IM ForceFieldFile is initalized from the current directory as im_database.h5


##### The compute function starts the construction of a new or given database. The construction run ends by confirming the qulity of the database and the result are stored.
- note that here we provide the constructed database ('im_database.h5') as the computational cost of the second derivatives are very demanding.

In [7]:
# Run the database consruction
dyn_drv.imforcefieldfile = 'im_database.h5'
im_results = dyn_drv.compute(molecule, basis)

Initial Density {(2, 3, 5, 9): {np.float64(0.0): 1, np.float64(36.0): 3, np.float64(72.0): 1, np.float64(108.0): 1, np.float64(144.0): 0, np.float64(180.0): 2, np.float64(216.0): 1, np.float64(252.0): 0, np.float64(288.0): 0, np.float64(324.0): 0}}
* Info * Using 6-31G* basis set for RESP charges...                                                                       
* Info * Using GAFF (v2.11) parameters.                                                                                   
         Reference: J. Wang, R. M. Wolf, J. W. Caldwell, P. A. Kollman, D. A. Case, J. Comput. Chem. 2004,
         25, 1157-1174.
                                                                                                                          
* Info * Updated bond angle 4-6-7 (cd-cd-cc) to 126.500 deg                                                               
* Info * Updated bond angle 5-4-6 (cc-cd-cd) to 126.544 deg                                                               
* Inf

KeyboardInterrupt: 

In [5]:
ff_gen = vlx.MMForceFieldGenerator()
ff_gen.create_topology(molecule)

* Info * Using 6-31G* basis set for RESP charges...                                                                       
* Info * Using GAFF (v2.11) parameters.                                                                                   
         Reference: J. Wang, R. M. Wolf, J. W. Caldwell, P. A. Kollman, D. A. Case, J. Comput. Chem. 2004,
         25, 1157-1174.
                                                                                                                          
* Info * Updated bond angle 4-6-7 (cd-cd-cc) to 126.500 deg                                                               
* Info * Updated bond angle 5-4-6 (cc-cd-cd) to 126.544 deg                                                               


### To use the constructed IM force field one has to initialize the InterpolationDriver() with user-defined settings (recommended). The current implementation is restricted to well seperated IM and MM regions and is not yet included into a linking atom framework.

In [6]:
z_matrix = dyn_drv.define_z_matrix(molecule)
im_driver = vlx.InterpolationDriver(z_matrix)
interpolation_settings = { 'interpolation_type':'shepard', 
                            'exponent_p':'2',
                            'exponent_q':'2', 
                            'confidence_radius':'0.5',
                            'imforcefieldfile':'im_database.h5'}
im_driver.update_settings(interpolation_settings)

openmmdyn = vlx.OpenMMDynamics()
openmmdyn.create_system_from_molecule(molecule, ff_gen, qm_atoms='all')
openmmdyn.run_qmmm(im_driver, im_driver)


* Info * Full molecule as QM region                                                                                       
* Info * QM region parameters written to qm_region.xml                                                                    
* Info * QM region: 0 ... 15                                                                                              
* Info * No external MM atoms found in the system                                                                         
* Info * System parameters written to qm_region_system.xml                                                                
* Info * System coordinates written to qm_region_system.pdb                                                               
QM/MM Simulation Parameters
QM Driver: IM Driver
Ensemble: NVE
Integration method: VerletIntegrator
Friction: 1.0 1/ps
Timestep: 0.5 fs
Total simulation time in ns: 0.0005
Step: 0 / 1000 Time: 0.0 ps
Potential Energy QM region: -2899550.05797694 kJ/mol
Potential

KeyboardInterrupt: 